<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN13/13.1-neural_style_transfer_with_gif_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Neural style transfer


Dieses Notizbuch enthält die Codebeispiele aus Kapitel 8, Abschnitt 3 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

Neben DeepDream gibt es noch eine weitere bedeutende Entwicklung bei auf Deep Learning beruhenden Bildverarbeitungsverfahren: den von *Leon Gatys et al.* im Sommer 2015 vorgestellten **Neural-Style-Algorithmus** zur Stilübertragung [1]. Der Algorithmus wurde kontinuierlich verfeinert, und seit der Erstveröffentlichung wurden viele Varianten entwickelt. Zudem gibt es diverse Smartphone-Apps, die ihn verwenden. Der Einfachheit halber beschränkt sich dieser Abschnitt auf die in der ursprünglichen Arbeit vorgestellte Variante.

Der Algorithmus überträgt den Stil eines Referenzbilds auf ein Zielbild und erhält dabei dessen Inhalt. 

[1] [Leon A. Gatys, Alexander S. Ecker und Matthias Bethge, A Neural Algorithm of Artistic Style,
arXiv (2015)](https://arxiv.org/abs/1508.06576).

## Stil und Inhalt

- Der **Stil** beschreibt in diesem Zusammenhang im Wesentlichen *Texturen, Farben und visuelle Muster auf verschiedenen räumlichen Skalen* eines Bilds. 
- Der **Inhalt** ezieht sich auf die allgemeinere Makrostruktur des Bilds. Inder obigen Abbildung gehören die blauen und gelben kreisförmigen Pinselstriche in Vincent van Goghs Gemälde Sternennacht zum Stil, die Gebäude in dem in Tübingen aufgenommenen Foto stellen hingegen den Inhalt dar.


Die eng mit der Texterzeugung verwandte Idee der Stilübertragung hat in der Bildverarbeitung eine lange Vorgeschichte, die viel weiter zurückreicht als die des erst 2015 entwickelten Neural-Style-Algorithmus. Wie sich herausstellte, lassen sich
mit auf Deep Learning beruhenden Implementierungen der Stilübertragung Ergebnisse erzielen, die alles, was mit klassischen Methoden des maschinellen Sehens erreicht werden kann, in den Schatten stellen. 

Und das löste eine erstaunliche Renaissance kreativer Anwendungen des maschinellen Sehens aus. Die entscheidende Idee bei der Implementierung der Stilübertragung liegt allen Deep-Learning-Algorithmen zugrunde: Man definiert eine Verlustfunktion, die festlegt, was erreicht werden soll, und minimiert sie. Im vorliegenden Fall ist klar, was erreicht werden soll: Der Inhalt des ursprünglichen Bilds soll erhalten werden, während gleichzeitig der Stil eines Referenzbilds übernommen wird. 


Der Prozess des Style Transfers ist in Abbildung 7 dargestellt. Um den Style Transfer durchzuführen,
werden drei Bilder benötigt. Das erste Bild enthält den gewünschten **Stil** $S$ *(Style)*,
das zweite enthält den **Inhalt** $C$ *(Content)* und bei dem dritten Bild handelt es sich um das
generierte Bild $x$ aus dem Stil-Bild und Inhalt-Bild. 

Das VGG19 dient beim Style Transfer als Funktion $f_w$, um Feature Maps zu generieren, die entweder dem `style` oder dem `content` entsprechen. Da der **Inhalt** (`content`) des Bildes erst in den letzten Schichten des Netzes erkannt wird, muss für die Berechnung der Verlustfunktion für den `content` ein Convolutional Layer verwendet werden, das sich im letzten Layer-Block des VGG19 befindet. 

Bei der Extraktion des **Stils** für kleine Muster werden Feature Maps aus den vorderen Schichten verwendet und Feature Maps aus den hinteren Schichten für grosse Muster. Das Netz wird nicht trainiert, die Gewichte des vortrainierten Netzes werden anfangs eingelesen und anschliessend nicht mehr verändert. Somit sind die Gewichte $w$ statisch. Um das zu generierende Bild $x$ zu berechnen, muss dieses zunächst initialisiert werden. In der vorliegenden Implementierung
wurde dies mit dem Inhalt-Bild $C$ gemacht, jedoch wäre auch weisses Rauschen möglich. Nachdem ein Bild mit VGG19 analysiert wurde, werden verschiedene Feature Maps von Convolutional Layern abgespeichert, die anschliessend für die Optimierung verwendet werden. 




## Zielfunktion $\mathcal{L}=\alpha\cdot \mathcal{L}_C+\beta \cdot \mathcal{L}_S$

Wenn wir Inhalt und Stil mathematisch definieren könnten, sähe eine passende zu minimierende
Verlustfunktion folgendermaßen aus:

```
loss = distance(style(reference_image) - style(generated_image)) +
       distance(content(original_image) - content(generated_image))
```

`distance` ist hier eine Normierungsfunktion wie die L2-Norm, `content` ist eine
Funktion, die eine Repräsentation des Bildinhalts errechnet, und `style` ist eine
Funktion, die eine Repräsentation des Stils liefert. 

- Die Minimierung dieser Verlustfunktion sorgt dafür, dass `style(generated_image)` und `style(reference_image)` bzw. `content(generated_image)` und `content(original_image)` von jeweils gleicher Grössenordnung sind, und erzielt so die definierte Stilübertragung.
- Gatys et al. machten die entscheidende Beobachtung, dass **tiefe neuronale Netze eine Möglichkeit bieten, die Funktionen `style` und `content` mathematisch zu definieren**. 

Sehen wir uns also an, wie das funktioniert.

## The content loss $\mathcal{L}_C$: Verlustfunktion für den Inhalt


Wie Sie bereits wissen, enthalten die Aktivierungen der ersten Layer eines NNs lokale Informationen über das Bild, die höher gelegenen Layer dagegen zunehmend *globale* und *abstrakte* Informationen. 

Mit anderen Worten: **Die Aktivierungen der verschiedenen Layer eines CNNs stellen eine Zerlegung des Inhalts eines
Bilds auf unterschiedlichen räumlichen Skalen dar.** Deshalb würde man erwarten, dass der eher globale und abstrakte Inhalt eines Bilds durch die Repräsentationen in den oberen Layern eines CNNs erfasst wird.

.

Aus diesem Grund ist die L2-Norm für den Abstand zwischen den Aktivierungen eines oberen Layers eines vortrainierten CNNs, berechnet für das Zielbild, und den Aktivierungen des gleichen Layers, berechnet für das erzeugte Bild, ein guter
Kandidat für die Verlustfunktion. Auf diese Weise ist sichergestellt, dass das erzeugte Bild aus Perspektive des oberen Layers Ähnlichkeit mit dem ursprünglichen Zielinhaltbild besitzt. Wenn die Annahme stimmt, dass die oberen Layer
eines CNNs tatsächlich den Inhalt ihrer Eingabebilder repräsentieren, sollte der Inhalt eines Bilds auf diese Weise erhalten bleiben.

Um die Optimierung durchzuführen, wird der Fehler berechnet, welcher zwischen dem generierten Bild $x$ und den vorgegebenen Bilder $C$ und $S$ entsteht. Der **Content-Loss** $\mathcal{L}_C$ wird *pixelweise* durch die Methode der kleinsten Quadrate nach folgender Formel berechnet.


$$\boxed{ \mathcal{L}_C = \sum_i^n \left[ f_w(C)-f_w(x) \right]^2} \tag{1}$$

## The style loss $\mathcal{L}_S$: Verlustfunktion für den Stil

Der Stil ist durch lokale Strukturen definiert. Da diese auf dem ganzen Bild vorkommen können,
muss der Stil *translationsinvariant* sein. Um dies zu erreichen, wird die **Gram-Matrix**
$G$ für das generierte Bild $x$ und das Stil-Bild $S$ aus den extrahierten Schichten berechnet.

Die Verlustfunktion für den Inhalt verwendet nur einen einzelnen oberen Layer,
die von Gatys et al. definierte Verlustfunktion für den Stil $\mathcal{L}_S$ verwendet jedoch mehrere Layer eines CNNs: Es wird versucht, das Aussehen des Referenzstilbilds nicht nur auf einer einzigen Skala, sondern auf allen vom CNN extrahierten räumlichen Skalen zu erfassen. Gatys et al. verwenden als Verlustfunktion die **Gramsche Matrix** $G$ der Aktivierungen eines Layers, also das Skalarprodukt der Feature-Maps eines Layers. Dieses Skalarprodukt repräsentiert gewissermaßen die Korrelationen zwischen den Merkmalen eines Layers. Diese Korrelationen erfassen eine Statistik der Muster auf einer bestimmten räumlichen Skala, die empirisch dem Erscheinungsbild der auf dieser Skala sichtbaren Texturen entsprechen.


Um die **Gram-Matrix** zu berechnen, werden die Feature Maps in hochdimensionale Vektoren
gewandelt, das heisst sämtliche Feature Maps einer Schicht werden zu einer Matrix zusammengefügt.
Diese Matrizen werden verwendet, um den mittleren quadratischen Abstand für
jede Schicht gemäss Formel (2) zu berechnen

$$\boxed{E_l=\frac{1}{4N_l^2\cdot M_l^2} \sum_{i,j} \left[ G(S)_{ij}-G(x)_{ij}\right]^2} \tag{2}$$

Dabei ist $N_l$ die Anzahl der Feature Maps und $M_l$ die Grösse der jeweiligen Feature Map. Der
Style-Loss $\mathcal{L}_S$ kann anschliessend durch Aufsummieren von $E_l$ unter Berücksichtigung der Gewichte $w_l$ berechnet werden:

$$\mathcal{L}_S = \sum_{l=0}^L w_l \cdot E_l \tag{3}$$

Die Verlustfunktion für den Stil hat also zum Ziel, ähnliche interne Korrelationen zwischen den Aktivierungen verschiedener Layer im Referenzstilbild beim erzeugten Bild zu erhalten. Umgekehrt ist dadurch gewährleistet, dass die auf verschiedenen
räumlichen Skalen erkennbaren Texturen des Referenzstilbilds im erzeugten Bild ähnlich aussehen.

## Der Gesamtverlust $\mathcal{L}$

Aus den beiden Formeln $\mathcal{L}_C$ (1) und $\mathcal{L}_S$ (2) kann nun der totale Fehler  berechnet werden:

## $$ \boxed{\mathcal{L} = \alpha \cdot \mathcal{L}_C + \beta \cdot \mathcal{L}_S}$$

Wir können somit **ein vortrainiertes CNN verwenden, um eine Verlustfunktion zu definieren**, die Folgendes leistet:
- Sie erhält den Inhalt, indem sie ähnliche Aktivierungen allgemeiner Layer des Zielinhaltbilds im erzeugten Bild übernimmt. Das CNN sollte feststellen, dass das Zielinhaltbild und das erzeugte Bild die gleichen Objekte enthalten.
- Sie erhält den Stil, indem sie ähnliche *Korrelationen der Aktivierungen* sowohl in tiefer liegenden als auch in höher liegenden Layern übernimmt. Merkmalskorrelationen erfassen Texturen: Das erzeugte Bild und das Referenzstilbild sollten auf verschiedenen räumlichen Skalen die jeweils gleichen Texturen enthalten.



<img src="Bilder/FastStyleTransfer_Johnson.PNG" width="840" align="center"/>

[3] [Johnson, J., Alahi, A. und Li, F.-F., “Perceptual Losses for Real-Time Style Transfer
and Super-Resolution”, CoRR, Jg. abs/1603.08155, 2016.](http://web.eecs.umich.edu/~justincj/papers/eccv16/JohnsonECCV16.pdf)

[4] [Johnson, J., Alahi, A. und Li, F.-F., “Perceptual Losses for Real-Time Style Transfer
and Super-Resolution: Supplementary Material”, CoRR, 2016.](http://web.eecs.umich.edu/~justincj/papers/eccv16/JohnsonECCV16Supplementary.pdf)


## Neural Style Transfer mit PyTorch

Beim Neural Style Transfer (NST) wird der **Inhalt eines Bildes** mit dem **künstlerischen Stil eines anderen Bildes** kombiniert. Die zugrunde liegende Idee stammt aus der Arbeit von *Gatys et al.* und nutzt ein vortrainiertes Convolutional Neural Network (CNN), in der Regel **VGG19**, um Stil- und Inhaltsinformationen aus Bildern zu extrahieren.

Das Verfahren nutzt nicht die Klassifikationsfähigkeit des Netzwerks, sondern die **Aktivierungen seiner Zwischenlayer**, um sowohl die **semantischen Inhalte** als auch die **stilistischen Eigenschaften** eines Bildes zu erfassen.

### Vorgehensweise in drei Schritten:

1. **Feature-Extraktion mit VGG19**  
   Ein tiefes CNN (hier: VGG19) wird verwendet, um die Aktivierungen der Zwischenlayer für drei Bilder zu berechnen:
   - das Content-Bild (Zielinhalt)
   - das Style-Bild (Stilvorlage)
   - das generierte Bild (wird trainiert)

2. **Definition der Verlustfunktion**  
   Die sogenannte Perceptual Loss setzt sich zusammen aus:
   - **Content Loss**: Vergleich der Feature-Maps zwischen generiertem und Content-Bild
   - **Style Loss**: Vergleich der Gram-Matrizen zwischen generiertem und Style-Bild
   - **TV Loss (optional)**: Glättung des Bildes durch Minimierung lokaler Unterschiede

3. **Optimierung des Bildes**  
   Das Zielbild wird pixelweise durch **Gradientenabstieg** so verändert, dass es gleichzeitig dem Inhalt des Content-Bildes und dem Stil des Style-Bildes entspricht.


> Damit das Verfahren gut funktioniert, sollten Content- und Style-Bild in ähnlicher Größe vorliegen. In der Regel wird die Höhe der Bilder auf z. B. **400 Pixel** skaliert, um Rechenzeit zu sparen und bessere Ergebnisse zu erzielen.

In [ ]:
# -----------------------------------------------
# Neural Style Transfer mit VGG19 (PyTorch)
# Basierend auf der offiziellen PyTorch-Tutorial-Implementierung
# -----------------------------------------------

import os
import urllib.request
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.models import vgg19, VGG19_Weights
from torchvision.transforms.functional import InterpolationMode


Zuerst wird geprüft, ob eine CUDA-fähige GPU verfügbar ist. Wenn ja, wird "cuda" als Gerät gewählt, sonst "cpu". Dieses Gerät wird dann als Standardgerät für alle folgenden Tensoroperationen gesetzt, sodass man nicht ständig .to(device) angeben muss. Zum Schluss wird das gewählte Gerät zur Kontrolle ausgegeben. Das sorgt für flexiblen, geräteunabhängigen Code, der sowohl auf GPU als auch CPU läuft.

In [ ]:
# -----------------------------------------------
# Geräteeinstellung & Default
# -----------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
print("Verwendetes Gerät:", device)


## Bildpfade & Download

### Was passiert hier?

1. **Ordner anlegen**  
   ```python
   os.makedirs("data/Bilder", exist_ok=True)
   ```
   Es wird ein Ordner `data/Bilder` erstellt, falls er noch nicht existiert. So wird sichergestellt, dass der Speicherort für die Bilder verfügbar ist.

2. **URLs und Pfade definieren**  
   Es werden zwei Bild-URLs festgelegt:
   - Ein **Content-Bild** (Porträtfoto einer Frau)
   - Ein **Style-Bild** (Gemälde von Ljubow Popowa)

   Zusätzlich werden die lokalen Speicherpfade definiert, unter denen diese Bilder abgelegt werden sollen:
   - `data/Bilder/portrait.jpg`
   - `data/Bilder/popova_style.jpg`

3. **User-Agent setzen**  
   Einige Server blockieren Anfragen ohne User-Agent. Daher wird ein Header gesetzt, um sich als normaler Browser auszugeben:
   ```python
   headers = {"User-Agent": "Mozilla/5.0"}
   ```

4. **Download-Funktion definieren**  
   Die Funktion `download_image(url, path)` lädt ein Bild von einer URL herunter und speichert es unter dem angegebenen Pfad.

5. **Bilder nur bei Bedarf herunterladen**  
   Vor dem Herunterladen wird überprüft, ob die Dateien bereits vorhanden sind. Falls nicht, werden sie heruntergeladen:
   ```python
   if not os.path.exists(content_path):
       download_image(content_url, content_path)
   ```

6. **Statusmeldung**  
   Am Ende wird eine Bestätigung ausgegeben:
   ```
   ✅ Bilder erfolgreich heruntergeladen und gespeichert.
   ```

### Zweck:
Dieser Code stellt sicher, dass die benötigten Bilder (Content & Style) lokal verfügbar sind, ohne sie jedes Mal neu herunterladen zu müssen. Das ist besonders hilfreich für Anwendungen wie **Neural Style Transfer**.

In [ ]:
import os
import urllib.request
from IPython.display import display
import ipywidgets as widgets

# -----------------------------------------------
# Bild-Download
# -----------------------------------------------
os.makedirs("data/Bilder", exist_ok=True)

content_url = "https://www.myposter.de/magazin/wp-content/uploads/2016/06/Portrait-Frau-lachend-shutterstock_381113020_kl.jpg"
style_url = (
    "https://upload.wikimedia.org/wikipedia/commons/7/78/Popova_Air_Man_Space.jpg"
)
content_path_default = "data/Bilder/portrait.jpg"
style_path_default = "data/Bilder/popova_style.jpg"

headers = {"User-Agent": "Mozilla/5.0"}


def download_image(url, path):
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as response, open(path, "wb") as out_file:
        out_file.write(response.read())


if not os.path.exists(content_path_default):
    download_image(content_url, content_path_default)
if not os.path.exists(style_path_default):
    download_image(style_url, style_path_default)

print("✅ Portrait- und Popova-Style-Bild erfolgreich geladen.")


In [ ]:
# -----------------------------------------------
# StyleTransfer-Bilder herunterladen
# -----------------------------------------------
target_dir = "data/Bilder/StyleTransfer"
os.makedirs(target_dir, exist_ok=True)

filenames = [
    "Klimt2.jpg",
    "Kopenhagen.jpg",
    "Mosaic2.jpg",
    "Sonnenblumen.jpg",
    "Van_Gogh.jpg",
    "Vassily_Kandinsky7.jpg",
    "candy.jpg",
    "composition-with-figures_popova.jpg",
    "composition.jpg",
    "graffiti.jpg",
    "popova.png",
    "portrait.jpg",
    "portrait_mann.jpg",
    "portrait_women.jpg",
    "simpsons.jpg",
    "udnie.jpg",
]

base_url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/data/StyleTransfer/"

for name in filenames:
    save_path = os.path.join(target_dir, name)
    if os.path.exists(save_path):
        continue
    try:
        urllib.request.urlretrieve(base_url + name, save_path)
    except Exception as e:
        print(f"❌ Fehler bei {name}: {e}")


In [ ]:
# -----------------------------------------------
# Bildpfade
# -----------------------------------------------
content_images = {
    "portrait": "data/Bilder/StyleTransfer/portrait.jpg",
    "Kopenhagen": "data/Bilder/StyleTransfer/Kopenhagen.jpg",
    "portrait_mann": "data/Bilder/StyleTransfer/portrait_mann.jpg",
    "portrait_women": "data/Bilder/StyleTransfer/portrait_women.jpg",
    "simpsons": "data/Bilder/StyleTransfer/simpsons.jpg",
}

style_images = {
    "popova": "data/Bilder/StyleTransfer/popova.png",
    "candy": "data/Bilder/StyleTransfer/candy.jpg",
    "Van_Gogh": "data/Bilder/StyleTransfer/Van_Gogh.jpg",
    "udnie": "data/Bilder/StyleTransfer/udnie.jpg",
    "Vassily_Kandinsky7": "data/Bilder/StyleTransfer/Vassily_Kandinsky7.jpg",
    "composition": "data/Bilder/StyleTransfer/composition.jpg",
    "composition_popova": "data/Bilder/StyleTransfer/composition-with-figures_popova.jpg",
    "graffiti": "data/Bilder/StyleTransfer/graffiti.jpg",
    "Mosaic2": "data/Bilder/StyleTransfer/Mosaic2.jpg",
    "Sonnenblumen": "data/Bilder/StyleTransfer/Sonnenblumen.jpg",
    "Klimt2": "data/Bilder/StyleTransfer/Klimt2.jpg",
}


In [ ]:
# -----------------------------------------------
# Dropdown-Menüs mit ipywidgets
# -----------------------------------------------
content_dropdown = widgets.Dropdown(
    options=content_images,
    description="Content:",
    layout=widgets.Layout(width="60%"),
    style={"description_width": "initial"},
)

style_dropdown = widgets.Dropdown(
    options=style_images,
    description="Style:",
    layout=widgets.Layout(width="60%"),
    style={"description_width": "initial"},
)

# Ausgabe-Label
output_widget = widgets.Output()  # Umbenennung, damit kein Konflikt entsteht

# Globale Variablen für die aktuellen Pfade
selected_content_path = content_dropdown.value
selected_style_path = style_dropdown.value


def update_paths(change=None):
    global selected_content_path, selected_style_path
    output_widget.clear_output()
    with output_widget:
        selected_content_path = content_dropdown.value
        selected_style_path = style_dropdown.value
        print("✅ Ausgewählte Pfade:")
        print("Content:", selected_content_path)
        print("Style:", selected_style_path)


# Verbindung der Dropdowns mit dem Callback
content_dropdown.observe(update_paths, names="value")
style_dropdown.observe(update_paths, names="value")

# Anzeige
display(content_dropdown, style_dropdown, output_widget)

# Initiales Anzeigen der Auswahl
update_paths()


## Bildverarbeitung

1. **Bildgröße festlegen**  
   Die Zielgröße für die Bilder wird abhängig vom Gerät gewählt:
   - **512 Pixel**, wenn eine GPU verfügbar ist (schnelleres Rechnen)
   - **128 Pixel**, wenn nur eine CPU verfügbar ist (ressourcenschonender)

2. **Transformationen definieren**  
   `loader` ist eine Pipeline zur Bildvorverarbeitung:
   - **Resize** auf die Zielgröße mit BICUBIC-Interpolation
   - **CenterCrop**, um das Bild quadratisch zuzuschneiden
   - **ToTensor**, um das PIL-Bild in ein PyTorch-Tensor (mit Werten in [0, 1]) umzuwandeln

3. **`unloader`**  
   Eine Umkehrfunktion: Wandelt ein Tensor-Bild wieder zurück in ein PIL-Bild zur Visualisierung.

4. **Funktion: `image_loader(image_path)`**  
   - Öffnet ein Bild von Pfad `image_path`
   - Konvertiert es zu RGB
   - Wendet die Transformationspipeline `loader` an
   - Fügt eine Batch-Dimension hinzu (`unsqueeze(0)`)
   - Gibt das Bild als Float-Tensor auf dem richtigen Gerät (`cpu` oder `cuda`) zurück

5. **Funktion: `imshow(tensor, title=None)`**  
   - Kopiert das Tensorbild zurück auf die CPU
   - Entfernt die Batch-Dimension
   - Wandelt es mit `unloader` in ein PIL-Bild um
   - Zeigt es mit `matplotlib` an
   - Optional kann ein Titel angegeben werden
   - Achsen werden ausgeblendet, und ein kurzes `pause()` sorgt dafür, dass das Bild korrekt dargestellt wird

### Zweck:
Dieser Block stellt sicher, dass Bilder korrekt geladen, skaliert und für die Verarbeitung (z. B. im Style Transfer) als Tensor vorbereitet werden – und später wieder anschaulich dargestellt werden können.

In [ ]:
# -----------------------------------------------
# Bildverarbeitung
# -----------------------------------------------
imsize = 512 if torch.cuda.is_available() else 128
loader = transforms.Compose(
    [
        transforms.Resize(imsize, interpolation=InterpolationMode.BICUBIC),
        transforms.CenterCrop(imsize),
        transforms.ToTensor(),
    ]
)

unloader = transforms.ToPILImage()


def image_loader(image_path):
    image = Image.open(image_path).convert("RGB")
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)


def imshow(tensor, title=None):
    image = tensor.cpu().clone().squeeze(0)
    image = unloader(image)
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis("off")
    plt.pause(0.001)


## Bilder laden

1. **Content- und Style-Bilder laden**  
   Mit der Funktion `image_loader(...)` werden die beiden vorbereiteten Bilder (Content und Style) eingelesen und in passende PyTorch-Tensoren umgewandelt:
   ```python
   content_img = image_loader(content_path)
   style_img = image_loader(style_path)
   ```
   Diese Bilder enthalten nun eine zusätzliche Batch-Dimension und liegen als 4D-Tensoren vor: `(1, 3, H, W)`.

2. **Größenvergleich**  
   Es wird geprüft, ob beide Bilder die gleiche Größe haben:
   ```python
   if content_img.size() != style_img.size():
       raise ValueError("Style und Content müssen gleiche Grösse haben!")
   ```
   Das ist notwendig, da der Style-Transfer-Prozess voraussetzt, dass Content- und Style-Bild dieselben Dimensionen besitzen – sonst funktionieren die Operationen (z. B. Feature-Extraktion oder Loss-Berechnung) nicht korrekt.

### Zweck:
Dieser Block lädt die beiden Hauptbilder für den Style Transfer und stellt sicher, dass sie korrekt und gleich groß sind – eine wichtige Voraussetzung für die Weiterverarbeitung im Modell.

In [ ]:
# -----------------------------------------------
# Bilder laden
# -----------------------------------------------
content_img = image_loader(selected_content_path)
style_img = image_loader(selected_style_path)

if content_img.size() != style_img.size():
    raise ValueError("Style und Content müssen gleiche Grösse haben!")


## Verlustfunktionen für Neural Style Transfer

In diesem Abschnitt werden zwei zentrale Verlustfunktionen definiert: eine für den **Content (Inhalt)** und eine für den **Style (Stil)**. Diese messen, wie gut das optimierte Bild dem Content- bzw. Style-Ziel entspricht.

### Übersicht der Funktionen und Klassen

| Name              | Typ         | Zweck                                                                                     |
|-------------------|--------------|--------------------------------------------------------------------------------------------|
| `ContentLoss`     | Klasse       | Misst den Unterschied zwischen den **Feature-Maps** des Content-Bildes und des generierten Bildes mithilfe von **MSE-Loss**. Nur der Inhalt wird berücksichtigt. |
| `gram_matrix`     | Funktion     | Berechnet die **Gram-Matrix**, welche die Korrelationen zwischen den Kanälen eines Features beschreibt. Sie ist die Grundlage zur Berechnung des Style-Loss. |
| `StyleLoss`       | Klasse       | Vergleicht die **Gram-Matrix** des Style-Bildes mit der des generierten Bildes mittels **MSE-Loss**. Dadurch wird der visuelle Stil übertragen. |


### Details zu den Komponenten

#### `ContentLoss`
- Speichert ein "Ziel"-Feature (`target`), das vom Content-Bild stammt.
- Im Forward-Pass wird der MSE-Loss zwischen dem aktuellen Input (aus dem generierten Bild) und dem Ziel berechnet.
- Gibt den Input unverändert zurück, damit das Bild weiter durch das Netz fließen kann.

#### `gram_matrix(input)`
- Formt das Input-Feature (Tensor der Form `[batch, channels, height, width]`) um in ein 2D-Feature.
- Multipliziert das Feature mit seiner Transponierten, um die Korrelationen zu berechnen.
- Normalisiert die Gram-Matrix, damit die Werte unabhängig von der Bildgröße sind.

#### `StyleLoss`
- Wandelt das Ziel-Feature des Style-Bildes in eine Gram-Matrix um.
- Im Forward-Pass wird die Gram-Matrix des Inputs berechnet und mit dem Ziel verglichen (MSE-Loss).
- Gibt ebenfalls den Input unverändert zurück.


### Zweck:
Diese Verlustfunktionen sind essenziell für den Style Transfer:
- Der **Content-Loss** bewahrt die Struktur und Formen des Ausgangsbildes.
- Der **Style-Loss** überträgt die Textur, Farbverteilung und Muster des Stilbildes.

In [ ]:
# -----------------------------------------------
# Verlustfunktionen
# -----------------------------------------------
class ContentLoss(nn.Module):
    def __init__(self, target):
        super().__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input


def gram_matrix(input):
    a, b, c, d = input.size()
    features = input.view(a * b, c * d)
    G = torch.mm(features, features.t())
    return G.div(a * b * c * d)


class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super().__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input


In [ ]:
# -----------------------------------------------
# Normalisierung (für VGG)
# -----------------------------------------------
cnn = vgg19(weights=VGG19_Weights.DEFAULT).features.eval()
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406])
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225])


class Normalization(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        self.mean = mean.view(-1, 1, 1)
        self.std = std.view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std


## Normalisierung (für VGG)

Beim Style Transfer wird ein vortrainiertes VGG19-Netzwerk verwendet. Dieses erwartet Eingabebilder, die mit bestimmten Mittelwerten und Standardabweichungen normalisiert wurden.


### Erklärung der Bestandteile

| Name                         | Typ       | Zweck                                                                                   |
|------------------------------|------------|------------------------------------------------------------------------------------------|
| `cnn`                        | Modell     | Enthält die **Feature-Extraktionsschichten** des vortrainierten VGG19-Modells (ohne Klassifikationskopf). |
| `cnn_normalization_mean`     | Tensor     | Mittelwerte der Farbkanäle (RGB), mit denen die Bilder für VGG normalisiert werden.     |
| `cnn_normalization_std`      | Tensor     | Standardabweichungen der Farbkanäle (RGB), für die Normalisierung der Eingabebilder.    |
| `Normalization`              | Klasse     | Modul zur Durchführung der Normalisierung im Forward-Pass des Netzwerks.                |


### Details

#### VGG-Modell
```python
cnn = vgg19(weights=VGG19_Weights.DEFAULT).features.eval()
```
- Lädt das **vortrainierte VGG19-Netzwerk** (ImageNet-Gewichte).
- `.features` extrahiert nur den **Feature-Teil** des Netzwerks (ohne die Klassifizierungsschichten).
- `.eval()` versetzt das Modell in den **Inferenzmodus**, d.h. ohne Dropout oder BatchNorm-Updates.

#### Mittelwert & Standardabweichung
```python
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406])
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225])
```
- Diese Werte stammen aus der **ImageNet-Datenverteilung** (Basis für das Training von VGG19).
- Sie werden benötigt, damit die Eingabebilder in der gleichen Weise normalisiert werden wie beim Training.

#### `Normalization`-Klasse
- Erbt von `nn.Module`.
- Die RGB-Kanal-Mittelwerte und Standardabweichungen werden so reshaped, dass sie auf Tensoren im Format `[B, C, H, W]` anwendbar sind.
- Im `forward(...)` wird das Bild kanalweise normalisiert


### Zweck:
Diese Normalisierung ist **essentiell**, damit die Bilder mit dem VGG-Netz korrekt verarbeitet werden. Ohne diese Anpassung würden die Features falsch interpretiert, was zu schlechten Ergebnissen beim Style Transfer führen würde.

In [ ]:
# -----------------------------------------------
# Modellaufbau mit Verlustschichten
# -----------------------------------------------
content_layers_default = ["conv_5", "conv_6"]
style_layers_default = ["conv_1", "conv_2", "conv_3", "conv_4"]


def get_style_model_and_losses(
    cnn,
    norm_mean,
    norm_std,
    style_img,
    content_img,
    content_layers=content_layers_default,
    style_layers=style_layers_default,
):
    normalization = Normalization(norm_mean, norm_std).to(device)
    content_losses, style_losses = [], []
    model = nn.Sequential(normalization)

    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = f"conv_{i}"
        elif isinstance(layer, nn.ReLU):
            name = f"relu_{i}"
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = f"pool_{i}"
        elif isinstance(layer, nn.BatchNorm2d):
            name = f"bn_{i}"
        else:
            raise RuntimeError(f"Unrecognized layer: {layer.__class__.__name__}")

        model.add_module(name, layer)

        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module(f"content_loss_{i}", content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module(f"style_loss_{i}", style_loss)
            style_losses.append(style_loss)

    # Modell kürzen
    for j in range(len(model) - 1, -1, -1):
        if isinstance(model[j], ContentLoss) or isinstance(model[j], StyleLoss):
            break
    model = model[: (j + 1)]
    return model, style_losses, content_losses


## Optimierung vorbereiten

Dieser Abschnitt definiert die Optimierungsmethode, mit der das Bild für den Style Transfer angepasst wird.


### Funktion: `get_input_optimizer(input_img)`

| Bestandteil                 | Bedeutung                                                                 |
|----------------------------|---------------------------------------------------------------------------|
| `input_img.requires_grad_()` | Aktiviert das **Gradient Tracking** für das Bild, da es optimiert werden soll. |
| `optim.LBFGS([...])`         | Verwendet den **L-BFGS-Optimierer** aus PyTorch, eine effektive Methode für kleine Probleme mit wenigen Parametern. |


### Zweck:
Beim Neural Style Transfer wird **nicht** das Netzwerk trainiert, sondern das **Eingabebild** (das zu stylende Bild) direkt optimiert.

Die Funktion gibt einen Optimierer zurück, der das `input_img` schrittweise verändert, sodass der **Style-Loss** und der **Content-Loss** minimiert werden.

Der **L-BFGS-Optimierer** eignet sich besonders gut für Style Transfer, da er konvergiert, ohne dass eine hohe Anzahl an Iterationen nötig ist.

In [ ]:
# -----------------------------------------------
# Optimierung vorbereiten
# -----------------------------------------------
def get_input_optimizer(input_img):
    return optim.LBFGS([input_img.requires_grad_()], lr=0.5)


## Style Transfer ausführen

In dieser Funktion wird der eigentliche **Style Transfer** durchgeführt: Das Eingabebild wird so optimiert, dass es den Inhalt des Content-Bildes und den Stil des Style-Bildes kombiniert.


### Funktion: `run_style_transfer(...)`

| Parameter         | Bedeutung                                                                 |
|-------------------|---------------------------------------------------------------------------|
| `cnn`             | Vortrainiertes VGG19-Netz (nur Feature-Teil)                             |
| `norm_mean/std`   | Mittelwert & Standardabweichung für VGG-Normalisierung                   |
| `content_img`     | Bild, dessen Inhalt beibehalten werden soll                              |
| `style_img`       | Bild, dessen Stil übertragen werden soll                                 |
| `input_img`       | Das zu optimierende Bild (initial meist eine Kopie von `content_img`)    |
| `num_steps`       | Anzahl der Optimierungsschritte (Standard: 500)                          |
| `style_weight`    | Gewichtung des Style-Loss (höher → mehr Stilübernahme)                   |
| `content_weight`  | Gewichtung des Content-Loss (höher → mehr Inhaltstreue)                  |


### Ablauf:

1. **Modell vorbereiten**  
   Die Hilfsfunktion `get_style_model_and_losses(...)` erstellt ein zusammengesetztes Modell mit eingefügten Style- und Content-Loss-Modulen.

2. **Optimierer initialisieren**  
   Das Eingabebild wird mit `get_input_optimizer(...)` als optimierbares Objekt vorbereitet (L-BFGS-Optimierer).

3. **Optimierungsschleife starten**  
   Für die festgelegte Anzahl an Schritten (`num_steps`) wird die folgende Funktion wiederholt:

   #### `closure()`:
   - Clamped (`input_img.clamp_(0, 1)`): Bild wird auf gültige Pixelwerte (0–1) begrenzt.
   - Optimierer-Zustand wird zurückgesetzt.
   - Forward-Pass durch das Modell → Verluste werden automatisch berechnet.
   - Style- und Content-Loss werden aufsummiert und gewichtet.
   - Backward-Pass: Gradienten werden berechnet.
   - Alle 50 Schritte wird der aktuelle Loss ausgegeben.

4. **Letztes Clamping**  
   Nach der Optimierung wird das finale Bild nochmals auf gültige Werte beschränkt.

5. **Ergebnis**  
   Die Funktion gibt das transformierte Bild zurück – eine Kombination aus Stil und Inhalt.


### Zweck:
Dies ist die zentrale Schleife, die den **Neural Style Transfer** durchführt. Sie verändert schrittweise das Eingabebild, bis es sowohl den gewünschten Stil als auch den gewünschten Inhalt bestmöglich wiedergibt.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import torch
import time
import os
from PIL import Image


def run_style_transfer(
    cnn,
    norm_mean,
    norm_std,
    content_img,
    style_img,
    input_img,
    num_steps=1000,
    style_weight=1e6,
    content_weight=1,
    show_every=100,  # Anzeige-Intervall (Bildanzeige)
    gif_every=20,  # GIF-Frame-Intervall (GIF speichern)
    content_path=None,
    style_path=None,
):
    print("Modell wird erstellt...")
    model, style_losses, content_losses = get_style_model_and_losses(
        cnn, norm_mean, norm_std, style_img, content_img
    )
    optimizer = get_input_optimizer(input_img)
    model.eval()

    print("Optimierung läuft...")

    # Fortschrittsanzeige vorbereiten
    progress_bar = widgets.IntProgress(
        value=0,
        min=0,
        max=num_steps,
        description="0%",
        bar_style="info",
        layout=widgets.Layout(width="100%"),
    )
    info_box = widgets.HTML(value="<b>Initialisierung läuft...</b>")
    metrics_box = widgets.HTML()

    display(
        widgets.VBox(
            [
                widgets.HTML(
                    "<h4 style='margin:0;'>🎨 <b>Style Transfer Fortschritt</b></h4>"
                ),
                progress_bar,
                info_box,
                metrics_box,
            ]
        )
    )

    # Log-Ausgabe vorbereiten
    log_output = widgets.Output(
        layout={"border": "1px solid #ccc", "height": "250px", "overflow_y": "auto"}
    )
    log_output.clear_output()
    display(
        widgets.VBox(
            [
                widgets.HTML(
                    "<h4 style='margin-top:30px;'>📜 <b>Iteration-Verlauf</b></h4>"
                ),
                log_output,
            ]
        )
    )

    start_time = time.time()
    run = 0
    frames = []
    last_style_loss = None
    last_content_loss = None

    while run <= num_steps:

        def closure():
            with torch.no_grad():
                input_img.clamp_(0, 1)
            optimizer.zero_grad()
            model(input_img)
            style_score = sum(sl.loss for sl in style_losses)
            content_score = sum(cl.loss for cl in content_losses)
            loss = style_score * style_weight + content_score * content_weight
            loss.backward()
            return loss

        optimizer.step(closure)
        run += 1
        progress_bar.value = run
        progress_bar.description = f"{int((run / num_steps) * 100)}%"

        # GIF-Frame speichern alle gif_every Iterationen
        if run % gif_every == 0 or run == num_steps:
            with torch.no_grad():
                input_img.clamp_(0, 1)
                img_clone = input_img.clone()
                img_np = img_clone.squeeze().cpu().numpy()
                img_np = img_np.transpose(1, 2, 0)
                img_uint8 = (img_np * 255).astype("uint8")
                frame = Image.fromarray(img_uint8)
                frames.append(frame)

        # Bildanzeige nur alle show_every Iterationen
        if run % show_every == 0 or run == num_steps:
            with torch.no_grad():
                input_img.clamp_(0, 1)
                style_score = sum(sl.loss for sl in style_losses)
                content_score = sum(cl.loss for cl in content_losses)

                style_diff = (
                    style_score.item() - last_style_loss
                    if last_style_loss is not None
                    else 0
                )
                content_diff = (
                    content_score.item() - last_content_loss
                    if last_content_loss is not None
                    else 0
                )
                last_style_loss = style_score.item()
                last_content_loss = content_score.item()

                elapsed = time.time() - start_time
                avg_time = elapsed / run
                eta = avg_time * (num_steps - run)

                # Metrik-Anzeige
                metrics_box.value = f"""
                <table style='font-size:14px;'>
                    <tr><td><b>🌀 Iteration:</b></td><td>{run} / {num_steps}</td></tr>
                    <tr><td><b>🎭 Style Loss:</b></td><td>{style_score.item():.2e} ({style_diff:+.2e})</td></tr>
                    <tr><td><b>🧠 Content Loss:</b></td><td>{content_score.item():.4f} ({content_diff:+.4f})</td></tr>
                    <tr><td><b>⏱️ Verstrichen:</b></td><td>{time.strftime("%H:%M:%S", time.gmtime(elapsed))}</td></tr>
                    <tr><td><b>📅 ETA:</b></td><td>{time.strftime("%H:%M:%S", time.gmtime(eta))}</td></tr>
                </table>
                """

                info_box.value = f"<b>🔁 Optimierung läuft – Iteration {run}</b>"

                # Log-Verlauf aktualisieren
                with log_output:
                    display(
                        HTML(f"""
                        <div style='font-family:monospace; border-bottom:1px solid #ddd; padding:4px 0;'>
                            <b>Iteration {run:>4}:</b> 
                            Style Loss = {style_score.item():.2e} ({style_diff:+.2e}), 
                            Content Loss = {content_score.item():.4f} ({content_diff:+.4f}), 
                            ⏱️ {time.strftime("%H:%M:%S", time.gmtime(elapsed))}
                        </div>
                    """)
                    )

                print(
                    f"Iteration {run}: Style Loss: {style_score.item():.2e}, Content Loss: {content_score.item():.4f}"
                )

                img_clone = input_img.clone()
                imshow(img_clone, title=f"Iteration {run}")

    progress_bar.bar_style = "success"
    progress_bar.description = "100%"
    total_time = time.time() - start_time

    info_box.value = "<b>✅ Style Transfer abgeschlossen!</b>"
    metrics_box.value += f"<br><b>⏳ Gesamtdauer:</b> {time.strftime('%H:%M:%S', time.gmtime(total_time))}"

    with torch.no_grad():
        input_img.clamp_(0, 1)

    def last_part(path):
        return os.path.splitext(os.path.basename(path))[0] if path else "unknown"

    content_name = last_part(content_path)
    style_name = last_part(style_path)
    gif_path = f"style_transfer_content-{content_name}_style-{style_name}.gif"

    if frames:
        frames[0].save(
            gif_path,
            save_all=True,
            append_images=frames[1:],
            duration=200,
            loop=0,
        )
        print(f"GIF gespeichert unter {gif_path}")

    return input_img




## Ausführung

In diesem letzten Schritt wird der gesamte Style-Transfer-Prozess gestartet und das Ergebnis visualisiert.


### Erklärung der Schritte:

| Codezeile                                      | Bedeutung                                                                 |
|------------------------------------------------|---------------------------------------------------------------------------|
| `input_img = content_img.clone()`              | Erzeugt eine Kopie des Content-Bildes als Startpunkt für die Optimierung. Das Eingabebild wird im Laufe des Style Transfers angepasst. |
| `output = run_style_transfer(...)`             | Startet den Style Transfer mit allen zuvor definierten Parametern. Das Ergebnis ist das optimierte Bild, das Stil und Inhalt vereint. |
| `plt.figure()`                                 | Öffnet eine neue Matplotlib-Figur für die Bildanzeige.                   |
| `imshow(output, title="Output Image")`         | Zeigt das Ergebnisbild im Plot mit dem Titel „Output Image“.             |
| `plt.ioff()` / `plt.show()`                    | Deaktiviert interaktive Anzeige und zeigt das Bildfenster.               |


### Zweck:
Das Bild `output`, das durch den Style Transfer entstanden ist, wird abschließend dargestellt. Es kombiniert den Inhalt des Content-Bildes mit dem Stil des Style-Bildes und stellt somit das Endergebnis des gesamten Prozesses dar.

In [ ]:
# -----------------------------------------------
# Ausführung
# -----------------------------------------------
plt.figure()
imshow(content_img, title="Content-Bild (Inhalt)")

plt.figure()
imshow(style_img, title="Style-Bild (Stil)")
input_img = content_img.clone()

output = run_style_transfer(
    cnn,
    cnn_normalization_mean,
    cnn_normalization_std,
    content_img,
    style_img,
    input_img,
    show_every=100,   # Bildanzeige alle 100 Iterationen
    gif_every=20,     # GIF-Frame alle 20 Iterationen
    content_path=selected_content_path,
    style_path=selected_style_path,
)

plt.figure()
imshow(output, title="Output Image")
plt.ioff()
plt.show()


Die obige Abbildung zeigt das Ergebnis. Was dieses Verfahren leistet, ist im Grunde genommen lediglich eine Form der Übertragung von Texturen eines Bilds auf ein anderes. 

- Am besten funktioniert es, wenn die Referenzstilbilder stark strukturiert und hochgradig selbstähnlich sind und die Zielinhaltbilder keine feinen Details enthalten müssen, um noch erkennbar zu sein. 
- Ziemlich abstrakte Merkmale, wie etwa der Stil eines Porträts, lassen sich für gewöhnlich nicht auf andere Bilder übertragen. 
- Der **Algorithmus gehört eher zur klassischen Signalverarbeitung als zur KI**, Sie dürfen hier also keine Wunder erwarten.
- Darüber hinaus arbeitet der Algorithmus zur Stilübertragung langsam. Die hier ausgeführte Transformation ist aber so einfach, dass sie auch von einem kleinen, schnellen Feedforward-Netz erlernt werden kann – sofern Ihnen geeignete Trainingsdaten zur Verfügung stehen. Schnelle Stilübertragungen können also erzielt werden, indem man zunächst einmal den Rechenaufwand investiert, um bei unverändertem Referenzstilbild eine Reihe von Ein-/Ausgabe-Trainingsbeispielen zu erzeugen, mit denen das einfache CNN trainiert wird, damit es diese stilspezifische Transformation erlernt. Sobald das erledigt ist, erfolgt die Stilübertragung auf ein Bild augenblicklich, denn es ist ja nur noch eine Zustandsweitergabe in
dem kleinen CNN nötig.




## Take aways

Bei der Stilübertragung wird ein neues Bild erzeugt, das den Inhalt eines Zielinhaltbilds
erhält und den Stil eines Referenzstilbilds übernimmt.
- Inhalte können von den höher liegenden Aktivierungen eines CNNs erfasst werden.
- Der Stil kann durch interne Korrelationen der Aktivierungen verschiedener Layer eines CNNs erfasst werden.
- Deep Learning ermöglicht es, eine Stilübertragung als Optimierungsaufgabe für ein vortrainiertes CNN mit Verlustfunktion zu formulieren.
- Von diesem grundlegenden Konzept sind viele Varianten und Verfeinerungen möglich.

## Where's the intelligence?

- Auch wenn Style-Transfer eher zur klassischen Signalverarbeitung gehört, gelicngt diese Übertragung von Mustern auf ein anderes Bild erst durch Deep Learning. Welche Eigenschaften von CNNs sind hier gefragt?